- Ollama -> See : https://ollama.com/download
  - For linux users :
    
    For this tutorial, run these instructions
    - Install Ollama : `curl -fsSL https://ollama.com/install.sh | sh`
    - Run the model : `ollama run mistral:7b-instruct-q4_0`
    
    -------------------------------------------------------------------------------
  
    Additional informations about Ollama
    - To remove Ollama : https://github.com/ollama/ollama/blob/main/docs/linux.md
    - To stop ollama server : `systemctl stop ollama`
    - To restart server : `systemctl start ollama`


In [1]:
from pathlib import Path

import pandas as pd
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_chroma import Chroma

In [2]:
ARTICLE_SOURCE_FILE_PATH = Path().resolve().parent /"data" / "malijet" / "source.csv"
CHROMA_DB_PERSIST_PATH = Path().resolve().parent / "data" / "chroma_db"
MODEL_NAME = "mistral:7b-instruct-q4_0"

In [3]:
llm = Ollama(model=MODEL_NAME)
llm

Ollama(model='mistral:7b-instruct-q4_0')

## Testing Simple LLM discussion

In [4]:
llm.invoke("Quelle est la recette la plus populaire en France ?")

'\nLa cuisine française est très riche et diverse, et il y a de nombreuses recettes qui sont populaires dans le pays. Mais si je dois choisir une seule recette, je dirais que la recette la plus populaire en France serait peut-être la boîte gorgonzola, aussi appelée "bœuf bourguignon". Il s\'agit d\'un plat simple mais très savoureux à base de filet mignon ou de bœuf bourguignon, cuit avec du vin rouge, des herbes et d\'une pointe de gorgonzola. Cet aliment est très apprécié dans les régions du Sud-Ouest, mais il est consommé partout en France.'

In [5]:
llm.invoke("Who is the most popular scientist in the world?")

"\nIt's difficult to definitively say who the most popular scientist in the world is, as popularity can be measured in various ways and scientists from different fields can gain popularity for their work in different regions of the world. However, as of 2021, some of the most well-known and widely recognized scientists globally include physicist Albert Einstein, biologist Charles Darwin, mathematician Isaac Newton, and astronaut Neil Armstrong. These scientists have made significant contributions to their respective fields and their work has had a profound impact on humanity."

In [6]:
llm.invoke("Where is the greatest place to visit in the world?")

"That's a difficult question as it depends on personal preferences. However, some popular places that are often considered the greatest to visit in the world include Paris, France for its romantic atmosphere and iconic landmarks; New York City, USA for its energy, diversity, and famous attractions; Tokyo, Japan for its blend of tradition and modernity; Sydney, Australia for its beautiful beaches and outdoor lifestyle; and Bali, Indonesia for its stunning rice terraces, temples, and serene natural beauty. Ultimately, the greatest place to visit in the world is subjective and depends on individual experiences and interests."

## 2. Build RAG with CSV file

In [7]:
loader = CSVLoader(
    file_path=ARTICLE_SOURCE_FILE_PATH,
    csv_args={
        "delimiter": "\t",
    }
)
loader

In [8]:
data = loader.load()
data[:3] # three first documents

[Document(page_content='title: Industrie : Le Président Assimi GOITA a reçu l’ancien footballeur international Seydou Keita\nsource_paper: Présidence\ndate: 2024-05-08\nlink: https://malijet.com/a_la_une_du_mali/290531-industrie--le-president-assimi-goita-a-recu-l’ancien-footballeur.html\ncontent: Le Président de la Transition, Son Excellence le Colonel Assimi GOÏTA, Chef de l’État, a reçu en audience, ce mardi 07 mai 2024, Seydou KEÏTA, ancien footballeur international et entrepreneur visionnaire, en prélude de la finalisation de son complexe industriel à Sanankoroba, dans le Cercle de Kati. Cette rencontre, qui s’est tenue en présence du ministre de l’Industrie et du Commerce, marque un jalon important dans le parcours de M. KEÏTA et pour le développement économique du Mali. Seydou KEÏTA, qui a entamé ce projet ambitieux en 2021, a transformé son rêve en réalité, illustrant parfaitement la transition réussie d’une carrière sportive à celle d’entrepreneur engagé. Le complexe industrie

In [9]:
# 100 articles as expected
len(data), pd.read_csv(ARTICLE_SOURCE_FILE_PATH, sep="\t").shape[0]

(100, 100)

## Let's see if the splitter is necessary

### Without splitter

In [10]:
def get_length_info(list_of_documents):
    ## search the content length statistics (nb characters)
    print('-'*10, "For character length", '-'*10)
    display(pd.Series([len(doc.page_content) for doc in list_of_documents]).describe())
    print('\n\n')
    
    ## search the content length statistics (nb words)
    print('-'*10, "For words length", '-'*10)
    display(pd.Series([len(doc.page_content.split(' ')) for doc in list_of_documents]).describe())

In [11]:
get_length_info(data)

---------- For character length ----------


count      100.000000
mean      3619.710000
std       3312.698748
min         78.000000
25%       1839.500000
50%       2738.500000
75%       4343.500000
max      20726.000000
dtype: float64




---------- For words length ----------


count     100.000000
mean      546.960000
std       534.128988
min         6.000000
25%       259.750000
50%       404.000000
75%       653.750000
max      3280.000000
dtype: float64

## 

## With splitter

In [12]:
## test the document splitter
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(documents=data)
len(documents)

169

In [13]:
get_length_info(documents)

---------- For character length ----------


count     169.000000
mean     2189.994083
std      1449.936741
min        78.000000
25%       642.000000
50%      2194.000000
75%      3922.000000
max      3999.000000
dtype: float64




---------- For words length ----------


count    169.000000
mean     331.082840
std      233.640659
min        6.000000
25%       86.000000
50%      322.000000
75%      599.000000
max      744.000000
dtype: float64

We move from 3280 words or tokens to 744 words or tokens. That's a division by 4 and might relevant for the final framework of RAG.

## Embeddings and Vector store

In [14]:
embeddings_llm = OllamaEmbeddings(model=MODEL_NAME)
embeddings_llm

OllamaEmbeddings(base_url='http://localhost:11434', model='mistral:7b-instruct-q4_0', embed_instruction='passage: ', query_instruction='query: ', mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None, show_progress=False, headers=None, model_kwargs=None)

In [15]:
# Embedding in vector space of 4096 as dimension
embeddings_llm.embed_query("Qui est le premier athlète de France")[:5], len(embeddings_llm.embed_query("Qui est le premier athlète de France"))

([-0.4267200231552124,
  5.168216705322266,
  3.9545648097991943,
  -2.950632095336914,
  6.3062944412231445],
 4096)

## Selection of Vector Store

Thanks to this brand new article from google, we can safely choose any open source Vector Store, make it available to Google NFS Filestore and access it easily through mounting filestore in Cloud Run (see section 3): https://cloud.google.com/blog/products/serverless/introducing-cloud-run-volume-mounts?hl=en

In [17]:
# Took 2h 52mn to execute, do not execute twice
# Database creation
# db = Chroma.from_documents(
#     documents=documents,
#     embedding=embeddings_llm,
#     persist_directory=CHROMA_DB_PERSIST_PATH.as_posix()
# )
# db

In [18]:
# load db from disk
db2 = Chroma(persist_directory=CHROMA_DB_PERSIST_PATH.as_posix(), embedding_function=embeddings_llm)
db2

In [19]:
llm.invoke("Tu connais le Mali ?")

"\nCet article sera écrit en anglais. Si vous préférez lire en français, voir notre autre article sur le Mali : <https://www.toutlemondeenligne.com/fr/mali>.\n\nThe Mali is a landlocked country located in West Africa. It has a population of approximately 18 million people and its official language is French, although Bambara, the most widely spoken local language, is also used in everyday life. The capital city of Mali is Bamako, which is located in the central part of the country.\n\nMali is known for its rich history and culture, as well as its natural beauty. Some popular attractions include the Dogon region, which is home to ancient rock formations and a unique way of life, and Timbuktu, an historic city that was once a major center of trade and learning in Africa.\n\nMali is also known for its beautiful landscapes, including vast savannas, rolling hills, and lush forests. Wildlife enthusiasts can enjoy activities such as bird watching, hiking, and camping in the country's national

## Retriever for RAG

In [20]:
retriever = db2.as_retriever(search_kwargs={"k": 3})
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7207d6ba3cd0>, search_kwargs={'k': 3})

In [21]:
query = "Que devons nous retenir du dialogue inter malien ?"
retriever.invoke(query), len(retriever.invoke(query))

([Document(page_content='de faire le consensus autour de l\'essentiel. L’UNTM, mal comprise, l\'a fait pour sauver le pays en Août 2022. Elle se situe encore dans le camp de la paix, ni pro, ni anti "Election" dans la confusion, la division, le chavirement de la nation comme les commanditaires européens sont arrivés à le faire en Haïti, en Somalie, au Congo de Patrice Lumumba, en Angola de Agostinho Neto. Aujourd\'hui, qu\'il s\'agisse de Wagner ou non le Mali a le droit pour sa survie de recourir à des forces paramilitaires ou à d\'autres épris de paix, de justice, de droit », a martelé Katilé. Enfin, il a souhaité l\'union sacrée à l’intérieur du Mali. Pour sa part, le ministre Fassoun Coulibaly a indiqué que le gouvernement est déterminé à poursuivre la mise en œuvre du plan d’actions 2023-2027 du pacte de stabilité sociale et de croissance signé le 25 août 2023. Après les interventions, il y a eu un impressionnant défilé des organisations et syndicats membre

In [22]:
# Prompt template
template = """
Réponds à la question uniquement grâce au contexte suivant et uniquement en langue française.
Si tu n'as pas de réponse explicite dans le contexte, réponds "Je n'ai pas assez d'informations pour répondre correctement".

Contexte : {context}

Question : {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [23]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [24]:
retriever.invoke("Qu'est ce que Assimi a fait ?")

[Document(page_content='title: Cnt-gate!!!!\nsource_paper: Le Challenger\ndate: 2024-04-29\nlink: https://malijet.com/a_la_une_du_mali/290275-cnt-gate!!!!.html\ncontent: L’arrêt rendu le 18 avril 2024 par la Cour constitutionnelle relatif à ‘’la loi organique portant indemnités, avantages et autres traitements des membres du Conseil national de transition (Cnt)’’ a suscité de nombreuses réactions au sein de la population, manifestement éprouvée par différentes crises. S’indignent et condamnent ceux qui estiment scandaleux que les conseillers nationaux s’accordent de tels privilèges dans le contexte actuel. Tentent aussi d’expliquer voire justifier ceux qui croient savoir pourquoi. En voici quelques-unes !  Les signataires de la déclaration commune du 31 mars 2024 : «Aucune reformulation du texte ne saurait innocenter le Cnt»    Dans une déclaration rendue publique le 26 avril 2024, les parties signataires de la déclaration commune du 31 mars 2024 «....constatent qu’au lieu 

In [25]:
chain.invoke("Qu'est ce que Assimi a fait ?")

"\nAssimi Goïta, le président malien, a été accusé d'avoir reçu l'ancien footballeur international Seydou Keita, qui était à l'origine de la création du Conseil national de transition (Cnt) en 2022. L'UNTM, une organisation militaire malienne, a été critiquée pour son rôle dans le renversement du gouvernement d'Alassane Ouattara et pour son soutien à Goïta. Des médias ont également noté que les trois pays de l'Alliance des États du Sahel avaient commencé à créer une stratégie d'autonomie énergétique, avec la présence de Russie en Afrique et la nouvelle donne militaire de Moscou après la fin de Wagner."

In [26]:
chain.invoke("Qui est le président de l'assemblée nationale ?")

"Je n'ai pas assez d'informations pour répondre correctement."

In [27]:
chain.invoke("Quelle célébrité le président a rencontré ?")

'\nLa Russie a fait beaucoup pour aider le Mali à "couper le cordon" avec l\'occupant.'

In [28]:
chain.invoke("What did the president say about technology?")

"Je n'ai pas assez d'informations pour répondre correctement à cette question, car elle n'est pas spécifiée dans le contexte fourni. Les articles mentionnés ne parlent pas de la technologie et son Excellence le colonel Assimi ne semble pas avoir abordé ce sujet lors de son discours."